# Denison CS181/DA210 Homework 4.a - Step 1

Before you turn this problem in, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells** (in the menubar, select Kernel$\rightarrow$Restart And Run All).

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

#### Import Python modules and load "SQL Magic"

In [1]:
import pandas as pd
import os
import os.path
import json
import sys
import importlib

module_dir = "../../modules"
module_path = os.path.abspath(module_dir)
if not module_path in sys.path:
    sys.path.append(module_path)

%load_ext sql

In [2]:
def getsqlite_creds(dirname=".",filename="creds.json"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the two parts needed for a connection string to
        a local provider using the "sqlite" dictionary within
        an outer dictionary.  
        
        Return a scheme and a dbfile
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D["sqlite1"]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]

In [3]:
scheme, dbdir, database = getsqlite_creds()
template = '{}:///{}/{}.db'
cstring = template.format(scheme, dbdir, database)
print("Connection string:", cstring)

Connection string: sqlite:///../../dbfiles/book.db


#### Establish Connection from Client to Server

In [4]:
%sql $cstring

---

## Part A: SQL Single-Table Selection and Row Filtering

In the following cells, your only action is to remove the `# YOUR CODE HERE` and `raise NotImplementedError()` lines, and then put a valid SQL statement as the **value** of string variable `query`.

In each case, when you execute the cell, the query will be sent to the database management system, a result obtained, and the result converted into a `pandas` data frame, whose prefix is shown.  (This allows for testing the results as well as displaying them in your Jupyter Notebook.)

_Note: you may want to view the `book` database in SQLiteStudio to get an idea of what the field names are as you work through these exercises._

**Q1:** Using the table `indicators`, write a query to find all unique years that appear in the table.

In [5]:
query = """SELECT DISTINCT year 
FROM indicators
"""

resultset1 = %sql $query
resultdf1 = resultset1.DataFrame()
resultdf1.head()

 * sqlite:///../../dbfiles/book.db
Done.


,year
0,1960
1,1961
2,1962
3,1963
4,1964


In [6]:
# Testing cell
assert len(resultdf1) == 59

**Q2:** Using the table `indicators`, write a query to find all rows (you can select all fields for these rows) with no missing data for `gdp`.

In [7]:
query = """SELECT * FROM indicators
WHERE gdp IS NOT NUll
"""

resultset2 = %sql $query
resultdf2 = resultset2.DataFrame()
resultdf2.head()

 * sqlite:///../../dbfiles/book.db
Done.


,year,code,pop,gdp,life,cell,imports,exports
0,1960,AFG,9.00,0.54,32.3,0.0,NaN,49.9
1,1960,AUS,10.28,18.58,70.8,0.0,2524.06,2022.9
2,1960,AUT,7.05,6.59,68.6,0.0,1408.80,1118.9
3,1960,BDI,2.80,0.20,41.3,0.0,NaN,NaN
4,1960,BEL,9.15,11.66,69.7,0.0,NaN,NaN


In [8]:
# Testing cell
assert len(resultdf2) == 9660

**Q3:** Using the table `indicators`, write a query to find all rows with no missing data for any of the numeric fields.

In [9]:
query = """SELECT * FROM indicators
WHERE year IS NOT NULL
AND pop IS NOT NULL
AND gdp IS NOT NULL
AND life IS NOT NULL
AND cell IS NOT NULL
AND exports IS NOT NULL
AND imports IS NOT NULL
"""


resultset3 = %sql $query
resultdf3 = resultset3.DataFrame()
resultdf3.head()

 * sqlite:///../../dbfiles/book.db
Done.


,year,code,pop,gdp,life,cell,imports,exports
0,1960,AUS,10.28,18.58,70.8,0.0,2524.06,2022.9
1,1960,AUT,7.05,6.59,68.6,0.0,1408.80,1118.9
2,1960,BEN,2.43,0.23,37.3,0.0,1.90,0.5
3,1960,BOL,3.66,0.37,42.1,0.0,71.10,67.9
4,1960,BRA,72.18,15.17,54.2,0.0,1586.75,1269.9


In [10]:
# Testing cell
assert len(resultdf3) == 6958

**Q4:** Using the table `indicators`, write a query to find all rows where exports is higher than imports. Select all fields for such rows.

In [11]:
query = """ SELECT * FROM indicators
WHERE exports > imports
"""


resultset4 = %sql $query
resultdf4 = resultset4.DataFrame()
resultdf4.head()

 * sqlite:///../../dbfiles/book.db
Done.


,year,code,pop,gdp,life,cell,imports,exports
0,1960,BRN,0.08,NaN,62.5,0.0,1.30,3.30
1,1960,CIV,3.50,0.55,36.9,0.0,119.92,151.19
2,1960,CMR,5.18,0.62,41.5,0.0,84.10,96.90
3,1960,DEU,72.81,NaN,69.3,0.0,10013.00,11381.00
4,1960,DOM,3.29,0.67,51.8,0.0,95.92,179.70


In [12]:
# Testing cell
assert len(resultdf4) == 2492
assert resultdf4.shape == (2492,8)

**Q5:** Using the table `indicators`, write a query to find the minimum non-zero number for `cell` that appears.  Although you could use `MIN`, you should try to do this using `ORDER BY` instead, so that the first row in your result has that minimum value.

In [13]:
query = """SELECT * FROM indicators
WHERE cell IS NOT NULL AND cell IS NOT 0
ORDER BY cell ASC
"""


resultset5 = %sql $query
resultdf5 = resultset5.DataFrame()
resultdf5.head()

 * sqlite:///../../dbfiles/book.db
Done.


,year,code,pop,gdp,life,cell,imports,exports
0,1981,JPN,117.65,1218.99,76.4,0.01,142869.0,151515.0
1,1982,DNK,5.12,60.41,74.6,0.01,16690.8,15394.9
2,1982,NOR,4.11,62.65,76.0,0.01,15465.9,17537.3
3,1985,ARE,1.37,40.60,69.8,0.01,6484.0,14043.0
4,1985,AUT,7.56,69.39,73.8,0.01,20967.3,17231.0


In [14]:
# Testing cell
assert resultdf5.loc[0,'cell'] == 0.01

**Q6:** Use a subquery to select the top ten entries in `indicators` with the highest population, then select the three from that group of ten that have the lowest GDP.  If you need to, review section 11.2.3 on subqueries.

In [15]:
query = """SELECT * FROM
(SELECT * FROM indicators
ORDER BY pop DESC
LIMIT 10)
ORDER BY gdp ASC
LIMIT 3
"""


resultset6 = %sql $query
resultdf6 = resultset6.DataFrame()
resultdf6.head()

 * sqlite:///../../dbfiles/book.db
Done.


,year,code,pop,gdp,life,cell,imports,exports
0,2017,IND,1338.66,2652.55,68.8,1168.90,442983.0,296212.0
1,2018,IND,1352.62,2726.32,NaN,NaN,NaN,NaN
2,2011,CHN,1344.13,7551.50,75.4,986.25,1741430.0,1899280.0


In [16]:
# Testing cell
assert len(resultdf6) == 3
assert resultdf6.loc[0,'gdp'] == 2652.55
assert resultdf6.loc[1,'pop'] == 1352.62

---

---

## Part B

**Q7:** How much time (in minutes/hours) did you spend on this homework assignment?

30 minutes

**Q8:** Who was your partner for this assignment?  If you worked alone, say so instead.

I worked alone